In [ ]:
import face_recognition
import cv2
import numpy as np
import pandas as pd
import os
from datetime import datetime, timedelta
import tkinter as tk
from tkinter import ttk, Label, Button, Canvas
from PIL import Image, ImageTk

# Global Variables
attendance_log = pd.DataFrame(columns=["Name", "Date", "Time"])
last_marked_time = {}  # Dictionary to track last attendance time for each user
TIME_INTERVAL = 1  # Default time interval in minutes
recognized_name = ""  # To display name when attendance is marked

# Function to load and encode faces from the dataset
def load_and_encode_faces(dataset_path):
    known_encodings = []
    known_names = []
    for student_folder in os.listdir(dataset_path):
        student_path = os.path.join(dataset_path, student_folder)
        if os.path.isdir(student_path):
            for image_name in os.listdir(student_path):
                image_path = os.path.join(student_path, image_name)
                try:
                    image = face_recognition.load_image_file(image_path)
                    face_encoding = face_recognition.face_encodings(image)
                    if face_encoding:
                        known_encodings.append(face_encoding[0])
                        known_names.append(student_folder)
                except Exception as e:
                    print(f"Error processing {image_path}: {e}")
    print("Face Encoding Complete.")
    return known_encodings, known_names

# Function to mark attendance
def mark_attendance(name):
    global last_marked_time, TIME_INTERVAL, recognized_name
    current_time = datetime.now()
    if name not in last_marked_time or (current_time - last_marked_time[name]) > timedelta(minutes=TIME_INTERVAL):
        current_date = current_time.strftime("%Y-%m-%d")
        formatted_time = current_time.strftime("%H:%M:%S")
        attendance_log.loc[len(attendance_log)] = [name, current_date, formatted_time]
        last_marked_time[name] = current_time
        recognized_name = f"Attendance Marked: {name}"  # Update display message
        update_attendance_table()
        print(f"Attendance marked for {name} at {formatted_time}")

# Update the attendance table in the GUI
def update_attendance_table():
    for row in attendance_table.get_children():
        attendance_table.delete(row)
    for _, row in attendance_log.iterrows():
        attendance_table.insert("", "end", values=list(row))

# Start the webcam for real-time face recognition
def start_webcam():
    global recognized_name
    video_capture = cv2.VideoCapture(0)
    print("Starting Face Recognition. Press 'q' to stop...")
    try:
        while True:
            ret, frame = video_capture.read()
            if not ret:
                break
            
            # Convert to RGB for face_recognition
            rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            face_locations = face_recognition.face_locations(rgb_frame)
            face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)
            
            # Reset recognized_name for the current frame
            recognized_name = ""

            # Compare detected faces with known faces
            for face_encoding, face_location in zip(face_encodings, face_locations):
                distances = face_recognition.face_distance(known_encodings, face_encoding)
                best_match_index = np.argmin(distances)
                
                name = "Unknown"
                if distances[best_match_index] < 0.5:  # Tighter tolerance
                    name = known_names[best_match_index]
                    mark_attendance(name)

                # Draw rectangle and name on the frame
                top, right, bottom, left = face_location
                cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
                cv2.putText(frame, name, (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
            
            # Convert for Tkinter
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            img = Image.fromarray(frame)
            imgtk = ImageTk.PhotoImage(image=img)
            video_label.imgtk = imgtk
            video_label.configure(image=imgtk)
            video_label.update()

            # Update attendance display
            attendance_message.configure(text=recognized_name)

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
    finally:
        video_capture.release()
        print("Webcam Stopped.")

# Save attendance to CSV
def save_attendance():
    attendance_log.to_csv("attendance_log.csv", index=False)
    print("Attendance saved to 'attendance_log.csv'")

# Load known faces
dataset_path = "./dataset"
known_encodings, known_names = load_and_encode_faces(dataset_path)

# Create the GUI
root = tk.Tk()
root.title("Face Recognition Attendance System")
root.geometry("900x600")
root.configure(bg="#f0f0f0")

# Outer Border
border_frame = Canvas(root, bg="purple", highlightthickness=0)
border_frame.pack(fill="both", expand=True, padx=10, pady=10)

# Split Left and Right Frames
left_frame = tk.Frame(border_frame, bg="#000", width=450, height=580)
left_frame.grid(row=0, column=0, padx=5, pady=5)
right_frame = tk.Frame(border_frame, bg="#ffffff", width=450, height=580)
right_frame.grid(row=0, column=1, padx=5, pady=5)

# Left Frame: Webcam Feed
video_label = Label(left_frame, bg="#000")
video_label.pack(fill="both", expand=True)

# Right Frame: Attendance Information
attendance_title = Label(right_frame, text="Attendance Log", font=("Helvetica", 16), bg="#ffffff")
attendance_title.pack(pady=5)

# Display Attendance Marked
attendance_message = Label(right_frame, text="", font=("Helvetica", 14), fg="green", bg="#ffffff")
attendance_message.pack(pady=5)

# Attendance Table
columns = ("Name", "Date", "Time")
attendance_table = ttk.Treeview(right_frame, columns=columns, show="headings", height=15)
for col in columns:
    attendance_table.heading(col, text=col)
attendance_table.pack(pady=10)

# Save Attendance Button
save_button = Button(right_frame, text="Save Attendance", command=save_attendance, font=("Helvetica", 12), bg="#2196F3", fg="white")
save_button.pack(pady=10)

# Start Webcam Button
start_button = Button(root, text="Start Webcam", command=start_webcam, font=("Helvetica", 14), bg="#4CAF50", fg="white")
start_button.pack(pady=10)

# Run the main loop
root.mainloop()